<a href="https://colab.research.google.com/github/Seongjin1225/AI_SCHOOL_9/blob/main/ML%26DL/12%EC%9B%94%2020%EC%9D%BC/12%EC%9B%94_20%EC%9D%BC_%EB%B3%B5%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMDB Dataset Load
from tensorflow.keras.datasets import imdb
(x_data_train, t_data_train), (x_data_test, t_data_test) = \
imdb.load_data(num_words=500)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_data_train_seq = pad_sequences(x_data_train, maxlen=100)
x_data_train_seq.shape

x_data_train_seq[0]

# 결국 one-hot encoding으로 처리해야 해요
# 하나하나의 token이 500개짜리 1차원 ndarray로 표현되어야
# 전체 -> (100,500)
from tensorflow.keras.utils import to_categorical
x_data_train_onehot = to_categorical(x_data_train_seq)
x_data_train_onehot.shape

(25000, 100, 500)

In [ ]:
## RNN 모델 구현
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(SimpleRNN(units=8,
                    activation='tanh',
                    input_shape=(100,500)))
model.add(Dense(units=1,
                activation='sigmoid'))


In [ ]:
# 전체 코드로 작성
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# 1. data loading
(x_data_train, t_data_train) , (x_data_test, t_data_test) = \
imdb.load_data(num_words=500)

# 2. 데이터 분리(test는 위에 있었으므로 valid data)
x_data_train, x_data_valid, t_data_train, t_data_valid = \
train_test_split(x_data_train,
                 t_data_train,
                 test_size=0.2,
                 stratify=t_data_train,
                 random_state=42)

# review의 길이 통일
x_data_train_seq = pad_sequences(x_data_train, maxlen=100)
x_data_valid_seq = pad_sequences(x_data_valid, maxlen=100)

# one-hot 형태로 변형
x_data_train_onehot = to_categorical(x_data_train_seq)
x_data_valid_onehot = to_categorical(x_data_valid_seq)

# 모델 생성
model = Sequential()

model.add(SimpleRNN(units=8,
                    activation='tanh',
                    input_shape=(100,500)))

model.add(Dense(units=1,
                activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss = 'binary_crossentropy',
              metrics=['acc'])

es_cb = EarlyStopping(monitor='val_loss',
                      patience=4,
                      restore_best_weights=True)

cp_cb = ModelCheckpoint(filepath='./best-simplernn-model.ckpt',
                        monitor='val_loss',
                        verbose=1,
                        save_best_only=True)

history = model.fit(x_data_train_onehot,
                    t_data_train,
                    epochs=100,
                    verbose=1,
                    batch_size=64,
                    validation_data = (x_data_valid_onehot, t_data_valid),
                    callbacks=[es_cb, cp_cb])

In [ ]:
# 시각화

train_loss = history.history['loss']
valid_loss = history.history['val_loss']

plt.plot(train_loss, 'o', color='r')
plt.plot(valid_loss, color='b')

plt.show()

In [ ]:
# one-hot 방식으로 하면 데이터의 양도 많아지고, 시간도 오래 걸림!
# word embedding 방식으로 구현

import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# 1. data loading
(x_data_train, t_data_train) , (x_data_test, t_data_test) = \
imdb.load_data(num_words=500)

# 2. 데이터 분리(test는 위에 있었으므로 valid data)
x_data_train, x_data_valid, t_data_train, t_data_valid = \
train_test_split(x_data_train,
                 t_data_train,
                 test_size=0.2,
                 stratify=t_data_train,
                 random_state=42)

# review의 길이 통일
x_data_train_seq = pad_sequences(x_data_train, maxlen=100)
x_data_valid_seq = pad_sequences(x_data_valid, maxlen=100)

# 모델 생성
model = Sequential()

model.add(Embedding(input_dim=500,  # vocabulary 수
                    output_dim=16,  # embedding size - 칸의 개수(one-hot의 경우 500개였음)
                    input_length=100))  # time-stamp(길이?)

model.add(SimpleRNN(units=8,
                    activation='tanh',
                    input_shape=(100,500)))

model.add(Dense(units=1,
                activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss = 'binary_crossentropy',
              metrics=['acc'])

es_cb = EarlyStopping(monitor='val_loss',
                      patience=4,
                      restore_best_weights=True)

cp_cb = ModelCheckpoint(filepath='./best-simplernn-model.ckpt',
                        monitor='val_loss',
                        verbose=1,
                        save_best_only=True)

history = model.fit(x_data_train_seq,
                    t_data_train,
                    epochs=100,
                    verbose=1,
                    batch_size=64,
                    validation_data = (x_data_valid_seq, t_data_valid),
                    callbacks=[es_cb, cp_cb])

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

sentences = [
    '강감찬은 나를 너무 너무 좋아해',
    '강감찬은 영화를 좋아해'
]

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)  # 단어사전 생성

tokenizer.word_index

word_encoding = tokenizer.texts_to_sequences(sentences)
word_encoding

[[1, 4, 2, 2, 3], [1, 5, 3]]

In [ ]:
# 단어사전에 없는 새로운 단어는??  -> 그냥 없애버리는 문제 발생
new_sentence = [
    '강감찬은 신사임당을 너무 좋아해'
]
new_word_encoding = tokenizer.texts_to_sequences(new_sentence)
new_word_encoding

# 해결방법?
# OOV(out of vocabulary) token으로 처리
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(sentences)

print(tokenizer.word_index)

new_word_encoding = tokenizer.texts_to_sequences(new_sentence)
new_word_encoding

# 길이도 맞춰보기
from tensorflow.keras.preprocessing.sequence import pad_sequences
padding = pad_sequences(word_encoding, maxlen=4)
padding

{'<OOV>': 1, '강감찬은': 2, '너무': 3, '좋아해': 4, '나를': 5, '영화를': 6}


array([[4, 2, 2, 3],
       [0, 1, 5, 3]], dtype=int32)

In [ ]:
# Naver 영화 리뷰 댓글 분석
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

# 데이터 가져오기
train_file = tf.keras.utils.get_file(
    '/content/ratings_train.txt',
    origin='https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt',
    extract=True
)
train_df = pd.read_csv(train_file, sep='\t')
train_df

test_file = tf.keras.utils.get_file(
    '/content/ratings_test.txt',  # 파일 저장 경로
    origin = 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt',  # origin => 데이터 읽어올 경로
    extract=True
)
test_df = pd.read_csv(test_file, sep='\t')
test_df.shape

4893335/4893335 [==============================] - 0s 0us/step


(50000, 3)

In [ ]:
# 한글 형태소 분석
# !pip install konlpy

from konlpy.tag import Okt, Kkma, Komoran

kkma = Kkma()
okt = Okt()
komoran = Komoran()

text = '이것은 소리없는 아우성. 저 푸른 해원을 향하여 흔드는 노스텔지어의 손수건'

kkma.morphs(text)
# ['이것', '은', '소리', '없', '는', '아우성', '.', '저', '푸른',
# '해원', '을', '향하', '여', '흔들', '는', '노스텔지어', '의', '손수건']

okt.morphs(text)
# ['이', '것', '은', '소리', '없는', '아우성', '.', '저', '푸른', '해원',
# '을', '향', '하여', '흔드는', '노스', '텔', '지', '어의', '손수건']

komoran.morphs(text)
# ['이것', '은', '소리', '없', '는', '아우성', '.', '저', '푸른', '해원',
# '을', '향하', '아', '흔들', '는', '노스', '텔', '짓', '어', '의', '손수건']

In [ ]:
# 데이터 전처리
# 1. 특수문자 제거(영문, 한글, 띄어쓰기 제외 삭제)
# 2. 결측치 제거
# 3. 불용어(stop word) 처리
# 4. 단어사전 생성 & 숫자로 변경하는 token화 작업
# 5. 동일한 문장 길이 설정

#################################################
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

# Naver 영화 리뷰 데이터를 로딩
train_file = tf.keras.utils.get_file(
    '/content/ratings_train.txt',
    origin='https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt',
    extract=True
)
train_df = pd.read_csv(train_file, sep='\t')

test_file = tf.keras.utils.get_file(
    '/content/ratings_test.txt',
    origin='https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt',
    extract=True
)
test_df = pd.read_csv(test_file, sep='\t')

# 1. 정규표현식 처리(^ == not)
train_df['document'] = train_df['document'].str.replace('[^A-Za-z가-힣ㄱ-ㅎㅏ-ㅣ ]','')
train_df.head()

# 2. 결측치 처리
train_df = train_df.dropna()
train_df.isnull().sum()

# 3. 불용어(stop word) 처리
# 관사, 전치사, 조사, 접속사등의 의미가 없는 단어 -> 불용어
# 여기서는 자주사용하는거 몇개만 처리
from konlpy.tag import Okt
okt = Okt()

def word_tokenization(text):
    stop_words=  ['는', '을', '를', '이', '가', '의', '던', '고', '하', '다',
                  '은', '에', '들', '지', '게', '도']
    return [x for x in okt.morphs(text) if x not in stop_words]

data_train = train_df['document'].apply(lambda x: word_tokenization(x))
data_train

In [ ]:
# 불용어 처리가 진행됬다고 가정하고 진행
# 4. 단어사전 만들고 문자 -> 숫자
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(data_train)

# 총 단어 수 확인
len(tokenizer.word_index)

# 각 토큰의 빈도수
tokenizer.word_counts

# 각 토큰의 빈도수를 숫자만 출력
tokenizer.word_counts.values()

# 단어 빈도수 5회 이상인 것만
def get_voca_size(threshold):
    count = 0
    for num in tokenizer.word_counts.values():
        if num > 5:
            count += 1
    return count

voca_size = get_voca_size(5)

# 단어사전 생성
tokenizer = Tokenizer(oov_token='<OOV>',
                      num_words=15000)

tokenizer.fit_on_texts(data_train)

# 각 문장을 숫자 벡터로 변환하여 encoding
data_train_seq = tokenizer.texts_to_sequences(data_train)


# 길이 다 맞춰주기
# 최대 길이 구하기
max_length = max([len(x) for x in data_train_seq])

from tensorflow.keras.preprocessing.sequence import pad_seqeuces
x_data_train = pad_sequences(data_train_seq,
                             maxlen=max_length)
t_data_train = train_df['label'].values

In [ ]:
# 모델 생성
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model = Sequential()

model.add(Embedding(input_dim=15000,
                    output_dim=32,
                    input_length=71))

model.add(Bidirectional(LSTM(units=16,
               activation='tanh')))

# hidden layer
model.add(Dense(units=32,
                activation='relu'))

# output layer
model.add(Dense(units=1,
                activation='sigmoid'))

model.compile(optimizer=Adam(learning_rate = 1e-4),
              loss='binary_crossentropy',
              metrics=['acc'])

# callback 설정
es_cb = EarlyStopping(monitor='val_loss',
                      patience=3,
                      restore_best_weights=True)

cp_cb = ModelCheckpoint(filepath='./best_model.ckpt',
                        save_weights_only=True,
                        save_best_only=True,
                        monitor='val_loss',
                        verbose=1)

# 학습
history = model.fit(x_data_train,
                    t_data_trian,
                    epochs=10,
                    verbose=1,
                    validation_split=0.2,
                    batch_size=64,
                    callbacks=[es_cb, cp_cb])

In [ ]:
# 학습이 끝난다음 예측
import pandas as pd
review_sentences = ['내가 만들어도 이것보단 잘 만들겠다.',
                    '너무너무 재미있었습니다. 감사합니다.',
                    '아..내 돈... 돈 아까워 죽을거 같아요',
                    '아나..이것도 영화라고 만들었냐. 무슨 스토리가 산으로 가냐',
                    '감동과 재미가 같이 있는 영화입니다. 훌륭합니다.',
                    '너무너무 재미없다.. 잠와 죽는줄.',
                    '너무너무 재미있다.. 잠이 확깨네.']

df = pd.DataFrame({'document':review_sentences})
df

# 정규표현식 사용해 특수분자 제거
df['document'] = df['document'].str.replace('[^A-Za-z가-힣ㄱ-ㅎㅏ-ㅣ ]','')
df

# 결측치 제거는 없으니까 생략

# 불용어 처리
from konlpy.tag import Okt
okt = Okt()

def stop_word(text):
    stop_word = ['는', '을', '를', '이', '가', '의', '던', '고', '하', '다',
                  '은', '에', '들', '지', '게', '도']

    return [x for x in okt.morphs(text) if x not in stop_word]

data_predict = df['document'].apply(lambda x : stop_word(x))

# 각 문장 숫자 벡터로 변환
data_predict_seq = tokenizer.texts_to_sequences(data_predict)

# 길이 통일
x_data_predict = pad_sequences(data_predict_seq,
                               truncating='post',  # maxlen 초과 시 앞에 잘라냄 의미
                               padding='pre',      # padding 시 앞에 채워넣는다 의미
                               maxlen=max_length)

# 예측
print(model.predict(x_data_predict))

<ipython-input-37-b688165b532d>:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df['document'] = df['document'].str.replace('[^A-Za-z가-힣ㄱ-ㅎㅏ-ㅣ ]','')


document    0
dtype: int64